In [26]:
import requests
from bs4 import BeautifulSoup
from helpers import wash_player_name
from config import data_positions_history
from urllib.parse import parse_qs
import pandas as pd
import json
import sys


class HistoryPage:
    def __init__(self, player, server, page=1):
        self.player = player
        self.server = server.title()
        self.page = page
        self.params = {'PlayerName': self.player, 'Gateway': self.server, 'PageNo': page}
        self.url = 'http://classic.battle.net/war3/ladder/w3xp-player-logged-games.aspx'
        self.servers = ['azeroth', 'lordaeron', 'northrend', 'kalimdor']
        self.soup = self.get_soup()
        self._validate()

    def __str__(self):
        return '{}@{}'.format(self.player, self.server)

    def __repr__(self):
        return '{}@{}'.format(self.player, self.server)

    def _validate(self):
        self.validate_server()
        self.validate_player()
        self.valiidate_page()

    def validate_player(self):
        error_span = self.soup.find('span', class_='colorRed')
        if error_span is not None:
            raise Exception('{} | Profile not found'.format(str(self)))

    def validate_server(self):
        if self.server.lower() not in self.servers:
            raise Exception('{} | Invalid server'.format(str(self)))

    def valiidate_page(self):
        if self.soup.find(text='Error Encountered'):
            raise Exception('{} | Invalid request'.format(str(self)))

    def get_soup(self):
        try:
            r = requests.get(self.url, params=self.params, timeout=2)
            r.raise_for_status()
            print(r.status_code)
        except requests.exceptions.RequestException as e:
            print(e)
            sys.exit(1)
        return BeautifulSoup(r.content, 'lxml')

    @property
    def game_containers(self):
        return self.soup.find('table', id='tblGames').find_all('tr', class_='rankingRow')

    @property
    def games(self):
        for game in self.game_containers:
            yield Game(self.player, game).parse()

    @property
    def next_page(self):
        url = self.soup.find(text='Next\xa0Page').parent.get('href')
        if url is not None:
            next_page = parse_qs(url).get('PageNo')[0]
            return int(next_page)


class Game:
    def __init__(self, player, soup):
        self.player = player
        self.soup = soup

    def parse(self):
        values = self.soup.find_all(['td'])
        values = [x.get_text().strip() for x in values]
        data = self.parse_values(values)
        game_id = self.soup.find_all(['td'])[0].a.get('href').split('&GameID=')[1]
        data['game_id'] = int(game_id)
        data['team_one'].append(self.player)

        for players in ['team_one', 'team_two']:
            data[players] = [wash_player_name(player) for player in data[players]]

        if data['winner'] == 'Win':
            data['winner'] = data['team_one']
        else:
            data['winner'] = data['team_two']

        return data

    @staticmethod
    def parse_values(values):
        data = {}
        for field, meta_data in data_positions_history.items():
            i = meta_data['position']
            v = values[i]
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            data[field] = value

        return data

if __name__ == '__main__':
    players = [
        {
            'player': 'Rellik',
            'server': 'northrend'
        }
    ]

    print('-- Testing --')
    for player in players:
        page = 1
        data_all = []

        while True:
                history_page = HistoryPage(player.get('player'), player.get('server'), page)
                print(history_page)
                data = list(history_page.games)
                data_all.extend(data)
                next_page = history_page.next_page
                if page >= next_page:
                        break
                page = next_page

        df = pd.DataFrame(data_all)
        print(df.shape)
        print(df.head(3))

    data = df.to_dict(orient='records')
    print(data)
    with open('./data_backfill/{}.json'.format(player.get('player')), 'w') as f:
        json.dump(data, f)


-- Testing --
HTTPConnectionPool(host='classic.battle.net', port=80): Read timed out. (read timeout=2)


SystemExit: 1

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [18]:
history_page.soup.find('table', id='tblGames')#.find_all('tr', class_='rankingRow')

In [23]:
history_page.soup.find('span', class_='colorRed')

In [25]:
if history_page.soup.find(text='Error Encountered'):
    

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<html>
<head>
<title>Warcraft III Ladder - Error</title>
<link href="/war3/includes/war3-human-ie.css" rel="stylesheet" type="text/css"/>
<link href="war3-ladder-reports.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#000000" leftmargin="0" marginheight="0" marginwidth="0" topmargin="0">
<table border="0" cellpadding="0" cellspacing="0" class="mainTable" height="100%" width="100%">
<tr height="75">
<td colspan="3" style="BACKGROUND-IMAGE: url('/war3/images/ladder-revise/left-bg.gif');  BACKGROUND-REPEAT: repeat-y; BACKGROUND-POSITION: left;" valign="top"><table background="/war3/images/ladder-revise/top-bg.gif" border="0" cellpadding="0" cellspacing="0" width="100%"><tr><td width="15"><img height="75" src="/war3/images/ladder-revise/top-left.gif" width="15"/></td><td align="center" width="100%"><a href="/war3/ladder/"><img border="0" height="75" src="/war3/images/ladder-revise/gatewayselect-header.gif" width

In [178]:
data_positions_history = {
    'date': {
        'position': 1,
        'function': lambda x: str(dateparser.parse(x).strftime("%Y-%m-%d %H:%M:%S"))
    },
    'game_type': {
        'position': 2,
        'function': lambda x: x.strip()
    },
    'map': {
        'position': 3,
        'function': lambda x: x.strip()
    },
    'team_one': {
        'position': 6,
        'function': lambda x: [] if x is '' else x.strip().split(',')
    },
    'team_two': {
        'position': 8,
        'function': lambda x: [] if x is '' else x.strip().split(',')
    },
    'game_length': {
        'position': 9,
        'function': lambda x: int(x.strip())
    },
    'winner': {
        'position': 10,
        'function': lambda x: x.strip()
    }
}

In [179]:
def wash_player_name(player):
    return player.replace('|c', '%7')

In [1]:
import requests
from bs4 import BeautifulSoup
from helpers import wash_player_name
from config import data_positions_history
from urllib.parse import parse_qs
import pandas as pd
import json


class HistoryPage:
    def __init__(self, player, server, page=1):
        self.player = player
        self.server = server.title()
        self.page = page
        self.params = {'PlayerName': self.player, 'Gateway': self.server, 'PageNo': page}
        self.url = 'http://classic.battle.net/war3/ladder/w3xp-player-logged-games.aspx'
        self.servers = ['azeroth', 'lordaeron', 'northrend', 'kalimdor']
        self.soup = self.get_soup()
        self._validate()

    def __str__(self):
        return '{}@{}'.format(self.player, self.server)

    def __repr__(self):
        return '{}@{}'.format(self.player, self.server)

    def _validate(self):
        self.validate_server()
        self.validate_player()

    def validate_player(self):
        error_span = self.soup.find('span', class_='colorRed')
        if error_span is not None:
            raise Exception('{} | Profile not found'.format(str(self)))

    def validate_server(self):
        if self.server.lower() not in self.servers:
            raise Exception('{} | Invalid server'.format(str(self)))

    def get_soup(self):
        try:
            r = requests.get(self.url, params=self.params)
        except requests.exceptions.RequestException as e:
            print(e)

        return BeautifulSoup(r.content, 'lxml')

    @property
    def game_containers(self):
        return self.soup.find('table', id='tblGames').find_all('tr', class_='rankingRow')[1:]

    def games(self):
        for game in self.game_containers:
            yield Game(self.player, game).parse()

    @property
    def next_page(self):
        url = self.soup.find(text='Next\xa0Page').parent.get('href')
        if url is not None:
            next_page = parse_qs(url).get('PageNo')[0]
            return int(next_page)


class Game:
    def __init__(self, player, soup):
        self.player = player
        self.soup = soup

    def parse(self):
        values = self.soup.find_all(['td'])
        values = [x.get_text().strip() for x in values]
        data = self.parse_values(values)
        game_id = self.soup.find_all(['td'])[0].a.get('href').split('&GameID=')[1]
        data['game_id'] = int(game_id)
        data['team_one'].append(self.player)

        for players in ['team_one', 'team_two']:
            data[players] = [wash_player_name(player) for player in data[players]]

        if data['winner'] == 'Win':
            data['winner'] = data['team_one']
        else:
            data['winner'] = data['team_two']

        return data

    @staticmethod
    def parse_values(values):
        data = {}
        for field, meta_data in data_positions_history.items():
            i = meta_data['position']
            v = values[i]
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            data[field] = value

        return data

if __name__ == '__main__':
    players = [
        {
            'player': 'Rellik',
            'server': 'northrend'
        }
    ]

    print('-- Testing --')
    for player in players:
        page = 1
        data_all = []

        while True:
                history_page = HistoryPage(player.get('player'), player.get('server'), page)
                data = list(history_page.games())
                data_all.extend(data)
                next_page = history_page.next_page
                if page >= next_page:
                        break
                page = next_page

        df = pd.DataFrame(data_all)
        print(df.shape)
        print(df.head(3))

    data = df.to_dict(orient='records')
    print(data)
    with open('./data_backfill/{}.json'.format(player.get('player')), 'w') as f:
        json.dump(data, f)


-- Testing --
(17, 8)
                  date   game_id  game_length game_type                map  \
0  2018-12-03 19:42:00  79822473            6      Solo  Plunder Isle_ L V   
1  2018-12-03 19:34:00  79822443            6      Solo         Echo Isles   
2  2018-12-02 20:57:00  79819025           11      Solo        Turtle Rock   

   team_one          team_two    winner  
0  [Rellik]           [Klapi]  [Rellik]  
1  [Rellik]  [GregvanAvermat]  [Rellik]  
2  [Rellik]         [FlyHigh]  [Rellik]  
[{'date': '2018-12-03 19:42:00', 'game_id': 79822473, 'game_length': 6, 'game_type': 'Solo', 'map': 'Plunder Isle_ L V', 'team_one': ['Rellik'], 'team_two': ['Klapi'], 'winner': ['Rellik']}, {'date': '2018-12-03 19:34:00', 'game_id': 79822443, 'game_length': 6, 'game_type': 'Solo', 'map': 'Echo Isles', 'team_one': ['Rellik'], 'team_two': ['GregvanAvermat'], 'winner': ['Rellik']}, {'date': '2018-12-02 20:57:00', 'game_id': 79819025, 'game_length': 11, 'game_type': 'Solo', 'map': 'Turtle Rock',

In [300]:
class Game:
    def __init__(self, player, soup):
        self.player = player
        self.soup = soup
        
    
    def parse(self):
        values = self.soup.find_all(['td'])
        values = [x.get_text().strip() for x in values]
        data = self.parse_values(values)
        id_ = self.soup.find_all(['td'])[0].a.get('href').split('&GameID=')[1]
        data['game_id'] = int(id_)
        data['team_one'].append(self.player)
        
        for players in ['team_one', 'team_two']:
            data[players] = [wash_player_name(player) for player in data[players]]
            
        if data['winner'] == 'Win':
            data['winner'] = data['team_one']
        else:
            data['winner'] = data['team_two']
            
        
        return data
     
    
    @staticmethod
    def parse_values(values):
        data = {}
        for field, meta_data in data_positions_history.items():   
            i = meta_data['position']
            v = values[i]
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            data[field] = value

        return data

In [301]:
history_page = HistoryPage('followgrubby', 'northrend')
print(history_page)

followgrubby@Northrend


In [302]:
data = list(history_page.games())
df = pd.DataFrame(data)
df

,date,game_id,game_length,game_type,map,team_one,team_two,winner
0,2018-12-01 19:18:00,79811310,9,Solo,Northern Isles,[followgrubby],[Fall3n],[followgrubby]
1,2018-11-28 02:46:00,79780015,8,Solo,Concealed Hill,[followgrubby],[4222222222222],[followgrubby]
2,2018-11-28 02:36:00,79779995,11,Solo,Last Refuge,[followgrubby],[iliketurtles],[followgrubby]
3,2018-11-25 02:14:00,79730307,15,Solo,Last Refuge,[followgrubby],[%700000000Mutti],[followgrubby]
4,2018-11-25 01:57:00,79730253,12,Solo,Last Refuge,[followgrubby],[GeniusOnWork],[followgrubby]
5,2018-11-25 01:40:00,79730203,23,Solo,Last Refuge,[followgrubby],[%700FF69B4Mutti],[followgrubby]
6,2018-11-25 01:11:00,79730096,15,Solo,Swamped Temple,[followgrubby],[d3r_schosch],[followgrubby]
7,2018-11-25 00:54:00,79730014,6,Solo,Terenas Stand_ L V,[followgrubby],[fcukingbullshyt],[followgrubby]
8,2018-11-25 00:44:00,79729968,17,Solo,Concealed Hill,[followgrubby],[TheSmurfer],[followgrubby]
9,2018-11-25 00:21:00,79729853,18,Solo,Echo Isles,[followgrubby],[Holly],[Holly]


In [304]:
page = 1
data_all = []

while True:
    history_page = HistoryPage('followgrubby', 'northrend', page)
    data = list(history_page.games())
    data_all.extend(data)
    next_page = history_page.next_page

    if page >= next_page:
        break
    page = next_page


In [305]:
df = pd.DataFrame(data_all)
df

,date,game_id,game_length,game_type,map,team_one,team_two,winner
0,2018-12-01 19:18:00,79811310,9,Solo,Northern Isles,[followgrubby],[Fall3n],[followgrubby]
1,2018-11-28 02:46:00,79780015,8,Solo,Concealed Hill,[followgrubby],[4222222222222],[followgrubby]
2,2018-11-28 02:36:00,79779995,11,Solo,Last Refuge,[followgrubby],[iliketurtles],[followgrubby]
3,2018-11-25 02:14:00,79730307,15,Solo,Last Refuge,[followgrubby],[%700000000Mutti],[followgrubby]
4,2018-11-25 01:57:00,79730253,12,Solo,Last Refuge,[followgrubby],[GeniusOnWork],[followgrubby]
5,2018-11-25 01:40:00,79730203,23,Solo,Last Refuge,[followgrubby],[%700FF69B4Mutti],[followgrubby]
6,2018-11-25 01:11:00,79730096,15,Solo,Swamped Temple,[followgrubby],[d3r_schosch],[followgrubby]
7,2018-11-25 00:54:00,79730014,6,Solo,Terenas Stand_ L V,[followgrubby],[fcukingbullshyt],[followgrubby]
8,2018-11-25 00:44:00,79729968,17,Solo,Concealed Hill,[followgrubby],[TheSmurfer],[followgrubby]
9,2018-11-25 00:21:00,79729853,18,Solo,Echo Isles,[followgrubby],[Holly],[Holly]


In [189]:
df.columns

Index(['date', 'game_length', 'game_type', 'id', 'map', 'team_one', 'team_two',
       'winner'],
      dtype='object')

,date,game_length,game_type,id,map,team_one,team_two,winner
0,2018-12-01 19:18:00,9,Solo,79811310,Northern Isles,[followgrubby],[Fall3n],[followgrubby]
1,2018-11-28 02:46:00,8,Solo,79780015,Concealed Hill,[followgrubby],[4222222222222],[followgrubby]
2,2018-11-28 02:36:00,11,Solo,79779995,Last Refuge,[followgrubby],[iliketurtles],[followgrubby]
3,2018-11-25 02:14:00,15,Solo,79730307,Last Refuge,[followgrubby],[%700000000Mutti],[followgrubby]
4,2018-11-25 01:57:00,12,Solo,79730253,Last Refuge,[followgrubby],[GeniusOnWork],[followgrubby]
5,2018-11-25 01:40:00,23,Solo,79730203,Last Refuge,[followgrubby],[%700FF69B4Mutti],[followgrubby]
6,2018-11-25 01:11:00,15,Solo,79730096,Swamped Temple,[followgrubby],[d3r_schosch],[followgrubby]
7,2018-11-25 00:54:00,6,Solo,79730014,Terenas Stand_ L V,[followgrubby],[fcukingbullshyt],[followgrubby]
8,2018-11-25 00:44:00,17,Solo,79729968,Concealed Hill,[followgrubby],[TheSmurfer],[followgrubby]
9,2018-11-25 00:21:00,18,Solo,79729853,Echo Isles,[followgrubby],[Holly],[Holly]


In [149]:
"""Scrapes a WC3 profile page from battlenet and returns json."""
from bs4 import BeautifulSoup
from botocore.vendored import requests
import dateparser
import pandas as pd
from config import data_positions


class Profile:
    def __init__(self, player, server):
        self.player = player
        self.server = server.title()
        self.params = {'PlayerName': self.player, 'Gateway': self.server}
        self.url = 'http://classic.battle.net/war3/ladder/w3xp-player-profile.aspx?'
        self.servers = ['azeroth', 'lordaeron', 'northrend', 'kalimdor']
        self.soup = self.get_soup()
        self._validate()
        self.tables = self._parse_tables()

    def __str__(self):
        return '{}@{}'.format(self.player, self.server)

    def get_soup(self):
        try:
            r = requests.get(self.url, params=self.params)
        except requests.exceptions.RequestException as e:
            print(e)

        return BeautifulSoup(r.content, 'lxml')

    def _validate(self):
        self.validate_server()
        self.validate_player()

    def parse(self):
        data = {}
        data['info'] = self.information
        data['individual'] = self.individual
        data['team'] = self.team
        return data

    def _parse_tables(self):
        soup = self.soup.find('table', class_='mainTable')
        tables = soup.find_all('td', {'align': 'center', 'valign': 'top'})
        tables = dict(zip(['info', 'individual', 'team'], tables))
        return tables

    @property
    def information(self):
        return {
            'player': self.player,
            'server': self.server,
            'clan': self.clan,
            'main_race': self.main_race,
            'home_page': self.home_page,
            'additional_info': self.parse_additional_info,
            'last_ladder_game': self.last_ladder_game
        }

    @property
    def individual(self):
        soup = self.tables.get('individual')
        type_ = 'individual'
        vocab = {'Team Games': 'random_team', 'Solo Games': 'solo', 'FFA Games': 'free_for_all'}
        data = {}

        for game_type, new_key in vocab.items():
            container = soup.find(text=game_type)
            if container is not None:
                table = container.parent.parent.parent.parent.parent
                values = [x.get_text() for x in table.find_all('b')]
                d = self.format_values(type_, values)
                d['win_percentage'] = self.calc_win_percentage(d['wins'], d['losses'])
                new_key = vocab[game_type]
                data[new_key] = d

        return data

    @property
    def team(self):
        soup = self.tables.get('team')
        type_ = 'teams'
        teams = soup.find_all(text='Partner(s):')
        data = []

        for team in teams:
            table = team.parent.parent.parent.parent.parent.parent.parent.parent
            values = self.extract_values(table)
            d = self.format_values(type_, values)
            d['win_percentage'] = self.calc_win_percentage(d['wins'], d['losses'])
            data.append(d)

        return data

    @property
    def home_page(self):
        soup = self.tables.get('info')
        home_page = soup.find('div', {'id': 'homePage'})
        home_page = home_page.b.get_text().strip()
        if home_page == '':
            return 'N/A'
        return home_page

    @property
    def parse_additional_info(self):
        soup = self.tables.get('info')
        additional_info_div = soup.find('div', {'id': 'additionalInfo'})
        script = additional_info_div.script.get_text()
        text_start = 'document.write("'
        i = script.find(text_start)
        if i == -1:
            additional_info = None
        else:
            i += len(text_start)

        text_end = '");'
        j = script[i:].find(text_end)
        additional_info = script[i: i + j]

        if additional_info in ['', None]:
            return 'N/A'
        return additional_info

    @property
    def last_ladder_game(self):
        soup = self.tables.get('info')
        last_ladder_game = soup.find(text='Last Ladder Game:')
        if last_ladder_game is not None:
            last_ladder_game = last_ladder_game.parent.parent.b.get_text()
        last_ladder_game = str(dateparser.parse(last_ladder_game).strftime('%Y-%m-%d'))
        return last_ladder_game

    @property
    def main_race(self):
        soup = self.tables.get('info')
        overall_stats_table = soup.find('td', class_='rankingHeader')
        if overall_stats_table is not None:
            overall_stats_table = overall_stats_table.parent.parent
        rows = overall_stats_table.find_all('tr')[1:-1]
        df = self.parse_stats_table(rows)
        if len(df[df['percentage_games'] >= 75]) != 0:
            main_race = df[df['percentage_games'] >= 75]['race'].values[0].title()
        else:
            main_race = 'No main race'

        return main_race

    @staticmethod
    def parse_stats_table(rows):
        data = []
        keys = ['race', 'wins', 'losses', 'win_percentage']

        for row in rows:
            row = [x.get_text().strip() for x in row.find_all('td')]
            row = dict(zip(keys, row))
            row['num_games'] = int(row['losses']) + int(row['wins'])
            data.append(row)

        df = pd.DataFrame(data)
        df['percentage_games'] = df['num_games'] * 100 / sum(df['num_games'])
        df['race'] = df['race'].apply(lambda x: x.lower().replace(':', ''))

        for col in ['wins', 'losses']:
            df[col] = df[col].astype(int)

        df['total_games'] = df['losses'] + df['wins']

        return df

    @property
    def clan(self):
        soup = self.tables.get('info')
        clan_url = soup.find('a', href=lambda x: x and x.startswith('w3xp-clan-profile.aspx?'))
        if clan_url is not None:
            return clan_url.get_text()
        return 'N/A'

    @staticmethod
    def format_values(type_, values):
        fields = ['wins', 'losses', 'partners', 'level', 'rank', 'experience']
        data = {}

        for field in fields:
            meta_data = data_positions[type_][field]
            if not meta_data:
                continue
            i = meta_data['position']
            v = values[i]
            if not v:
                continue
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            else:
                value = v

            data[field] = value

        return data

    @staticmethod
    def calc_win_percentage(wins, losses):
        win_percentage = round((100 * int(wins)) / (int(wins) + int(losses)), 2)
        return '{0:.2f}%'.format(win_percentage)

    @staticmethod
    def extract_values(table):
        values = []
        values_old = table.find_all('b')

        for i, value in enumerate(values_old):
            if i == 3:
                partners = [x.get_text() for x in value]
                if len(partners) > 1:
                    partners.remove('')
                values.append(partners)
            else:
                values.append(value.get_text())

        return values

    def validate_player(self):
        error_span = self.soup.find('span', class_='colorRed')
        if error_span is not None:
            raise Exception('{} | Profile not found'.format(str(self)))

    def validate_server(self):
        if self.server.lower() not in self.servers:
            raise Exception('{} | Invalid server'.format(str(self)))

    def request_solo(self):
        data = self.individual.get('solo')
        if data is not None:
            data.update({
                'main_race': self.main_race,
                'player': self.player,
                'server': self.server,
            })
        return data

    def request_random_team(self):
        data = self.individual.get('random_team')
        if data is not None:
            data.update({
                'main_race': self.main_race,
                'player': self.player,
                'server': self.server,
            })
        return data

    def request_info(self):
        data = self.information
        if data is not None:
            return data

if __name__ == '__main__':
    players = [
        {
            'player': 'romantichuman',
            'server': 'northrend'
        },
        {
            'player': 'pieck',
            'server': 'northrend'
        },
        {
            'player': 'pieck',
            'server': 'azeroth'
        },
        {
            'player': 'wearefoals',
            'server': 'northrend'
        },
        {
            'player': 'wearefoals',
            'server': 'azeroth'
        },
        {
            'player': 'wearefoals',
            'server': 'fake_server'
        },
        {
            'player': 'fake_player',
            'server': 'azeroth'
        },
        {
            'player': 'moon1234',
            'server': 'northrend'
        },
    ]
    print('-- Testing --')
    for player in players:
        print()
        try:
            profile = Profile(**player)
            print(profile)
            print(profile.parse())
        except Exception as e:
            print(e)


-- Testing --

romantichuman@Northrend
{'info': {'player': 'romantichuman', 'server': 'Northrend', 'clan': 'N/A', 'main_race': 'Human', 'home_page': 'twitch.tv/ToD', 'additional_info': 'twitter.com/YoanMerlo', 'last_ladder_game': '2018-12-01'}, 'individual': {'random_team': {'wins': 32, 'losses': 18, 'level': 10, 'rank': 'Unranked', 'experience': 2115, 'win_percentage': '64.00%'}, 'solo': {'wins': 1895, 'losses': 262, 'level': 47, 'rank': 'Rank 7', 'experience': 21229, 'win_percentage': '87.85%'}}, 'team': [{'wins': 2, 'losses': 2, 'partners': ['Lado'], 'level': 6, 'rank': 'Unranked', 'win_percentage': '50.00%'}, {'wins': 1, 'losses': 1, 'partners': ['123456789012345', 'LadoBlanco'], 'level': 5, 'rank': 'Rank 500', 'win_percentage': '50.00%'}, {'wins': 6, 'losses': 0, 'partners': ['LUL'], 'level': 11, 'rank': 'Unranked', 'win_percentage': '100.00%'}]}



KeyboardInterrupt: 